# Rode wijn in Portugal

Voor de Data Science casusopdracht heeft onze groep de rode wijn data set als onderzoekopdracht. In het eerste gedeelte van dit document zal er een een verkenning van de data plaatsvinden, waardoor er gerichte en interessante onderzoeksvragen opgesteld kunnen worden.

## Verkenning

Als eerste stap van de verkenning zal er onderzoek gedaan worden naar de huidige velden en de mogelijke verbeteringen aan de data set.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
redwine = pd.read_csv('redwine.csv', sep=';')
redwine.head()

In [ ]:
redwine.set_index('id', inplace=True)

In [ ]:
# Eerst eens bekijken hoeveel records we op dit moment hebben
len(redwine)

In [ ]:
# Bekijken hoeveel records het scheelt met ontbrekende waardes
len(redwine.dropna())

Hier zit best een fors verschil tussen, waardoor er met meer detail naar de data gekeken moet worden.

Daarnaast valt het ons op dat de country en variety velden allemaal gelijke data bevatten, waardoor deze irrelevant zijn.

In [ ]:
redwine.drop(['country', 'variety'], axis=1, inplace=True)
redwine.head()

Hieronder tellen we het aantal rijen met missende data per kolom.

In [ ]:
redwine.isna().sum()

Het valt op dat alle missende data voorkomt bij designation en price, echter zijn er ook een aantal kolommen die niet het goede type zijn. Deze zullen zeer waarschijnlijk dus ook data bevatten die niet als float gecast kan worden.

In [ ]:
redwine.dtypes

In [ ]:
redwine[np.isnan(pd.to_numeric(redwine["density"], errors="coerce"))]

Hieruit blijkt dat er maar 1 rij is waar de density niet klopt, omdat het er maar 1 is, zullen we deze verwijderen uit de dataset.

In [ ]:
redwine.drop(1475, inplace=True)
redwine["density"] = redwine["density"].astype("float")

De andere velden met het type object bevatten meer invalide waardes:

In [ ]:
print(len(redwine[np.isnan(pd.to_numeric(redwine["citric acid"], errors="coerce"))]))
print(len(redwine[np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]))

Omdat het bij alcohol ook om maar een aantal waardes gaat, is er gekozen om ook voor de records met ongeldige alcohol waardes de records te verwijderen.

In [ ]:
redwine = redwine[~np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]
redwine["alcohol"] = redwine["alcohol"].astype("float")

### Tweede dataset

Als tweede dataset is er een set gepakt welke ook de zelfde chemische samenstellingen bevat als de huidige set, alleen is deze gericht op witte wijnen. 

In [ ]:
whitewine = pd.read_csv('winequality-white.csv', sep=';')
whitewine.head()

In tegenstelling tot bij de rode wijn, is er niet veel informatie over de regio en naamgeving van de wijn beschikbaar. Daarnaast is het een kwaliteitscijfer en niet een waardering van een reviewer, waardoor de cijfers niet direct te vergelijken zijn.

Bij de witte wijn set worden alle kolommen wel correct ingelezen:

In [ ]:
whitewine.dtypes


#### Histogrammen

Om meer inzicht te krijgen in de verdelingen van de stoffen, is voor elke stof een histogram opegsteld. Ook voor de pH, quality en alcohol waardes zijn grafieken gemaakt.

In [ ]:
sns.set()
for column in whitewine.columns.values:
    plt.title(column)
    if column in redwine:
        if redwine[column].dtype != "object":
            plt.hist(redwine[column], color="#ff335577")
    plt.hist(whitewine[column], color="#ffff3377")
    plt.show()

Het valt hierbij op dat de quality en citric acid geen rode wijnen heeft, omdat de quality niet aanwezig is als veld in de dataset, en omdat citric acid van het type object is. In tegenstelling tot alcohol en density is hier nog geen oplossing voor, omdat er veel meer datapunten missen.

Daarnast is het interessant om te zien dat veel van deze grafieken geen goeie normaalverdeling vormen, wat doet vermoeden dat er een directe aanleiding is voor de verdeling van de stoffen. De grafiek van de pH waardes lijkt nog het meest normaal verdeeld.

## Onderzoeksvragen

Een eis voor het voltooien van deze casus is het opstellen (en beantwoorden) van onderzoeksvragen. Hierbij moet er rekening gehouden worden met de data die is aangeleverd en moet er een combinatie gemaakt worden met een externe set van data.

In het vorige gedeelte is een externe set verkend, welke betrekking heeft op witte wijnen in plaats van rode.

Met deze twee datasets en alle gegevens erin zijn we tot de volgende onderzoeksvragen gekomen:

- Welke proefpersonen geven gemiddeld meer of minder punten voor een wijn en hoeveel is dit?
- Zijn er groepen te onderscheiden in de wijn op basis van de stoffen en de wijnmakerijen?
- In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?
- Wat zijn de verschillen in verhoudingen van de chemische kenmerken bij rode t.o.v. witte wijnen?
